In [27]:
from math import ceil
import time, re, ast, sys, urllib, random, string, requests
import pandas as pd
from bs4 import BeautifulSoup
from random import randint
from selenium.webdriver.firefox.webdriver import FirefoxProfile
from urllib.request import urlopen, Request
from urllib.error import HTTPError, URLError
from itertools import cycle
from header_list import user_agent_list
from proxies_list import proxies_list_
from INITIAL_SCRAPPER_FUNCTIONS import *
from PARSER_FUNCTIONS import *
from LIST_DF_FUNCTIONS import *
from SEARCH_URL_GEN import *
from GET_SEARCH_URL import *
from GET_RESULTS import *
ua = user_agent_list
proxies = proxies_list_

In [49]:
cities_url = pd.read_csv('../Data/city_url_search_list.csv', header=None)
city_url_list = cities_url[0].tolist()
# areas_list = [num for num in areas if str(num) != 'nan']
city_url_list = random.sample(city_url_list, len(city_url_list))

In [50]:
under_prp_list = []
prp_list = []

In [30]:
# url = 'https://www.zillow.com/homes/for_sale/Fremont-CA/fsba,fsbo,new_lt/house,condo,apartment_duplex,townhouse_type/11540_rid/1-_baths/20000-_price/83-_mp/50-_size/1_pnd/priced_sort'

In [ ]:
def areas_extractor_url(proxies, prp_list, url_list, ua):
    for url in url_list:
        try:
            proxy = random.sample(proxies, 1)[0]
            print(proxies.index(proxy))
            print(proxy)
            start_time = time.time()
            soup = session_creator(proxy, ua, url)
            time.sleep(random.uniform(1.5,3.8))
            all_count = soup.findAll('meta', {'name': 'description'})
            print(all_count)
            print(time.time() - start_time)
            prp_list.append(all_count)
            url_list.remove(url)
            print(len(url_list))
            print(len(prp_list))
        except:
            print("Skipping. Connnection error")
            proxies.remove(proxy)
            print(len(proxies))
            return prp_list, url_list
    return prp_list, url_list


prp_list, city_url_list = areas_extractor_url(
    proxies, prp_list, city_url_list, ua)

2
50.224.173.189:8080
[<meta content="Zillow has 749 homes for sale in Hialeah FL. View listing photos, review sales history, and use our detailed real estate filters to find the perfect place." name="description"/>]
4.0389933586120605
63
247
36
209.97.233.3:61268
[<meta content="Zillow has 202 homes for sale in Burbank CA. View listing photos, review sales history, and use our detailed real estate filters to find the perfect place." name="description"/>]
4.3722779750823975
62
248
37
73.229.18.160:41062
[<meta content="Zillow has 440 homes for sale in Poughkeepsie NY. View listing photos, review sales history, and use our detailed real estate filters to find the perfect place." name="description"/>]
8.907651424407959
61
249
32
47.210.133.169:43305
[<meta content="Zillow has 1,420 homes for sale in Montgomery AL. View listing photos, review sales history, and use our detailed real estate filters to find the perfect place." name="description"/>]
10.026840448379517
60
250
8
64.19.116.82:6

In [58]:
def session_creator(proxy, ua, url):
    header = random.sample(ua, 1)[0]
    session = requests.Session()
    session.proxies = {"http": proxy, "https": proxy}
    req = session.get(url, headers=header)
    soup = BeautifulSoup(req.text, 'lxml')
    return soup

In [ ]:
def zip_prop_count(zip_list, proxies, prp_list, ua, ezl):
    proxy = random.sample(proxies, 1)[0]
    print(proxies.index(proxy))
    print(proxy)
    for num in zip_list:
        url = 'https://www.redfin.com/zipcode/' + \
            str(num) + '/filter/property-type=house+condo+townhouse,' + \
            'include=sold-1yr,min-price=100k,min-baths=1,include=sold-1yr'
        try:
            start_time = time.time()
            soup = session_creator(proxy, ua, url)
            print(num)
            print(len(zip_list))
            all_count = soup.findAll('div', {'class': 'homes summary'})
            if len(str(all_count)) >= 20:
                print(all_count)
                print(time.time() - start_time)
                ezl.append(num)
                prp_list.append(all_count)
                zip_list.remove(num)
                print(len(zip_list) + len(prp_list))
            else:
                print("Captcha!!!!!")
        except:
            print("Skipping. Connnection error")
            proxies.remove(proxy)
            print(len(proxies))
            return prp_list, zip_list, proxies, ezl
    return prp_list, zip_list, proxies, ezl

In [ ]:
def each_page(proxy, ua, url):
    soup = session_creator(proxy, ua, url)
    # start_time = time.time()
    time.sleep(random.uniform(0, 1) * 3)
    # print(time.time() - start_time)
    full_soup = soup.findAll('a', {'class': 'bottom link-override'})
    full_address = [fas['title'] for fas in full_soup]
    home_link = ['https://www.redfin.com' +
                 str(hls.get('href')) for hls in full_soup]
    df = {'full_address': full_address, 'home_link': home_link}
    return df

In [ ]:
def links_for_props(proxies, url_list, main_df, ua):
    proxy = random.sample(proxies, 1)[0]
    print(f'proxy number: {proxy}')
    i = randint(0, (len(url_list) // 2))
    print(f'starting from url number: {i}')
    while i < len(url_list):
        url = url_list[i]
        try:
            b = random.uniform(0.75, 2.25)
            time.sleep(b)
            # start_time = time.time()
            # print(f'total left: {len(url_list)}')

            data = each_page(proxy, ua, url)
            df = pd.DataFrame(data)
            eds = {'full_address': [], 'home_link': []}
            if data['full_address'] != eds['full_address']:
                main_df = pd.concat([main_df, df])
                url_list.pop(i)
                # a = (time.time() - start_time) * len(url_list)
                # print('SUCCESS!!')
                # print(f'Currently on url number: {i}')
                # print(f'time taken: {a/len(url_list)}')
                if i > 0:
                    i -= randint(0, 1)
                else:
                    i += 0
            else:
                # print('No results')
                # print(f'Currently on url number: {i}')
                i += randint(1, 5)
        except:
            # print("Skipping. Connnection error")
            proxies.remove(proxy)
            # print(f'proxies left: {len(proxies)}')
            # print(f'total left: {len(url_list)}')
            return url_list, main_df, proxies
    return url_list, main_df, proxies

In [ ]:
url = 'https://www.zillow.com/homedetails/895-Yakima-Dr-Fremont-CA-94539/25028455_zpid/?fullpage=true'

In [ ]:
header = random.sample(ua, 1)[0]
soup = session_creator(ua, url)

In [ ]:
a = soup.findAll('div', {'class': 'fact-container'})
b = soup.findAll('div', {'class': 'fact-value'})

In [ ]:
z = []
y = []

In [ ]:
for num in a:
    z.append(num.text)
for num in b:
    y.append(num.text)

In [ ]:
z = [num.replace('  ','').replace('   ','').replace('\r','').replace('\n','').replace('\xa0','') for num in z]

In [ ]:
l = [re.findall(r'(.+)\:', num) for num in z]
# print(l)
k = [re.findall(r'\:(.+)', num) for num in z]
# print(k)
nd = {}
i = 0
while i < len(l):
    if len(l[i])>0:
        if l[i][0] in nd.keys():
            nd[l[i][0]].extend(k[i])
        else:
            nd[l[i][0]] = k[i]
        i+=1
    else:
        i+=1
for k,v in nd.items():
    nd[k] = ', '.join([num for num in v])
print(nd)

In [ ]:
y = [num.replace('  ','').replace('   ','').replace('\r','').replace('\n','').replace('\xa0','') for num in y]

In [ ]:
dict(zip(z,y))